In [1]:
import numpy as np

import decimal
import matplotlib.pyplot as plt
import matplotlib.cbook as cbook
import subprocess
import math

In [2]:
# pequeña funcion que nos servira mas adelante para saber por cuanto debemos multiplicar
# el paso para poder obtener un numero entero que ingresar en el for
def num_after_point(x):
    s = str(x)
    if not '.' in s:
        return 0
    return len(s) - s.index('.') - 1

In [45]:
def chissqr(sigma_s):    # funcion que, dado un sigma sistematico, encuentra el chi cuadrado del grafico de residuos  

# creamos un nuevo archivo .tim en el cual a los errores sumamos este sigma sistematico

    timing_folder = './timing_' + str(sn) + '/'
    timfile = "data_" + str(sigma_s) + ".tim"     # nombre del nuevo archivo .tim, donde hemos sumado el sigma sistematico
    resfile = "residues_" + str(sigma_s) + ".dat" # nombre del archivo que contendra el chi cuadrado
    
    f = open(timing_folder + timfile, "w+")                    # abrimos el nuevo archivo .tim
    print("FORMAT 1" , file = f)               # escibimos la primera linea
             
# ingresamos los datos, con el error aumentado en una cantidad igual a sigma sistematico
    
    for j in range(len(error)):
        newerror = error[j] + sigma_s
#        f.write( str(file[j]) + ' ' + '{:11.6f}'.format(freq[j]) + ' ' + str(MJD[j]) + '   ' + '{:5.3f}'.format(newerror) + '  ' + str(telescope[j]))
        print( str(file[j]) + ' ' + '{:11.6f}'.format(freq[j]) + ' ' + str(MJD[j]) + '   ' + '{:5.3f}'.format(newerror) + '  ' + str(telescope[j]), file = f)
    
    f.close()
    
# llamamos a tempo2
    
    subprocess.call( "tempo2 >" + resfile + " -us -f J0437-4715.par " + timfile, cwd=timing_folder, shell = True)

# abrimos el archivo generado por tempo2, donde se encuentra el chicuadrado que buscamos

    chi_sqr_red = np.genfromtxt ( timing_folder + resfile, comments="none", dtype=float, skip_header=19, max_rows=1, usecols=(8) )

# devolvemos la informacion al programa principal

    return chi_sqr_red

In [52]:
# leemos los parámetros del archivo de entrada

xmin = np.genfromtxt("interval.txt", comments="none", dtype=float, skip_header=0, max_rows=1, usecols=(2) )
xmax = np.genfromtxt("interval.txt", comments="none", dtype=float, skip_header=1, max_rows=1, usecols=(2) )
step = np.genfromtxt("interval.txt", comments="none", dtype=float, skip_header=2, max_rows=1, usecols=(2) )
tol = np.genfromtxt("interval.txt", comments="none", dtype=float, skip_header=3, max_rows=1, usecols=(2) )
sn = np.genfromtxt("interval.txt", comments="none", dtype=int, skip_header=4, max_rows=1, usecols=(2) )

print("xmin = " + str(xmin))
print("xmax = " + str(xmax))
print("step = " + str(step))
print("tol = " + str(tol))
print("sn = " + str(sn))

a = num_after_point(step)           # número de decimales en el paso
xmin_int = int( xmin * (10 ** a) )  # límite inferior entero del intervalo
xmax_int = int( xmax * (10 ** a) )  # límite superior entero del intervalo
step_int = int( step * (10 ** a) )  # paso entero

n = int( (xmax_int-xmin_int)/step_int ) # numero de subintervalos

# creamos un array para que guarde los sigma sistematicos de prueba y otro para los chi_sqr

trial_sigma = np.empty(n)
chi_sqr = np.empty(n)

xmin = 0.0
xmax = 1.0
step = 0.1
tol = 0.0001
sn = 80


In [53]:
# leemos los errores en el TOA del archivo .tim generado por pat

timfile = "./timing_files/timing_" + str(sn) + ".tim"

file = np.loadtxt( timfile , skiprows=1 , dtype=str , usecols=(0) )
freq = np.loadtxt( timfile, skiprows=1, dtype=float , usecols=(1) )
MJD = np.loadtxt( timfile , skiprows=1 , dtype=decimal.Decimal , usecols=(2) )
error = np.loadtxt( timfile , skiprows=1 , dtype=float,  usecols=(3) )
telescope = np.loadtxt( timfile, skiprows=1, dtype=str, usecols=(4) )

#print(file)
#for i in range(0, len(error)):
#    print(file[i], freq[i], MJD[i], error[i], telescope[i])

In [54]:
# Set-up a period grid with values separated by 0.1 µs between 0 and 1

output = open("sigmasis_50.dat", "w")                                    # abrimos un archivo de salida

while step > 0.0001:

    i = 0
    for x in range(xmin_int, xmax_int, step_int):

        sigma_s = float(x)/ (10. ** a)      # sigma sistematico    

# guardamos la informacion

        trial_sigma[i] = sigma_s
        chi_sqr[i] = chissqr(sigma_s)
    
        print( str(trial_sigma[i]) + ' ' + str(chi_sqr[i]) )                   # escribimos por pantalla  
        output.write( str(trial_sigma[i]) + ' ' + str(chi_sqr[i]) )           # escribimos en el archivo de salida
    
        i += 1

# Creamos un nuevo array que sea la diferencia entre los chi_sqr obtenidos y 1
# y buscamos el chi cuadrado más cercano a uno y el correspondiente sigma

    diff_chi_sqr = abs ( chi_sqr - 1. )                   # creamos un array que sea la diferencia entre los chi y 1
    closer_chi = chi_sqr[ np.argmin(diff_chi_sqr)]        # hallamos el chi cuadrado más cercano a 1
    closer_sigma = trial_sigma[ np.argmin(diff_chi_sqr) ] # hallamos el correspondiente sigma 
    
# creamos un nuevo intervalo más acotado donde buscar el sigma sistematico

        
    if closer_chi < 1.0:
        xmax = closer_sigma
        xmin = closer_sigma - step
    elif closer_chi > 1.0:
        xmax = closer_sigma + step
        xmin = closer_sigma
    else:
        print("end of iterations")
        print("el chi_sqr mas cercano a 1 es: " + str( closer_chi ) )
        print("el correspondiente sigma_s es: " + str( closer_sigma ) )
        exit()        
    
    step = step / 10.0
    
    a = num_after_point(step)           # número de decimales en el paso
    xmin_int = int( xmin * (10 ** a) )  # límite inferior entero del intervalo
    xmax_int = int( xmax * (10 ** a) )  # límite superior entero del intervalo
    step_int = int( step * (10 ** a) )  # paso entero

    n = int( (xmax_int-xmin_int)/step_int ) # numero de subintervalos

# creamos un array para que guarde los sigma sistematicos de prueba y otro para los chi_sqr

    trial_sigma.resize(n)
    chi_sqr.resize(n)
    
# Hallamos el máximo chi_sqr mas cercano a 1 y el correspondiente sigma sistematico

    print("el chi_sqr mas cercano a 1 es: " + str( closer_chi ) )
    print("el correspondiente sigma_s es: " + str( closer_sigma ) )

0.0 11.6526
0.1 4.99658
0.2 2.81753
0.3 1.816
0.4 1.2701
0.5 0.939025
0.6 0.722837
0.7 0.573776
0.8 0.466606
0.9 0.386949
el chi_sqr mas cercano a 1 es: 0.939025
el correspondiente sigma_s es: 0.5
0.4 1.2701
0.41 1.22962
0.42 1.19107
0.43 1.15431
0.44 1.11924
0.45 1.08575
0.46 1.05376
0.47 1.02316
0.48 0.993895
0.49 0.965872
el chi_sqr mas cercano a 1 es: 0.993895
el correspondiente sigma_s es: 0.48
0.47 1.02316
0.471 1.02018
0.472 1.01721
0.473 1.01425
0.474 1.0113
0.475 1.00837
0.476 1.00545
0.477 1.00254
0.478 0.999647
0.479 0.996765
el chi_sqr mas cercano a 1 es: 0.999647
el correspondiente sigma_s es: 0.478
